In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import os
import wfdb
import random
import datetime
import time
import itertools
import pickle
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
from tempfile import TemporaryFile

from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, GRU
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import BatchNormalization
from keras.layers import Embedding

from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import LeaveOneGroupOut, GroupShuffleSplit
from sklearn.model_selection import GroupKFold
from operator import itemgetter

In [4]:
#from matplotlib import pyplot

def losscurve(historyv, name='losscurve'):
    loss = historyv.history['loss']
    val_loss = historyv.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'y', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.savefig(name)
    plt.legend()
    
    plt.show()

    
def acccurve(historyv, name='acccurve'):
    acc = historyv.history['accuracy']
    val_acc = historyv.history['val_accuracy']
    loss = historyv.history['loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, acc, 'y', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(name)
    plt.show()

def ROC_curve(test, pred_pr, name='ROC_curve'):
    auc = roc_auc_score(test, pred_pr)
    fpr, tpr, thresholds = roc_curve(test, pred_pr)
    
    plt.figure(figsize=(8,6))
    plt.plot([0, 1], [0, 1], 'k--', linewidth=3) # dashed line with black(k) color

    plt.plot(fpr, tpr, label='AUC = %0.4f' % auc, linewidth=3)

    plt.xlabel('False positive rate', fontsize=18)
    plt.ylabel('True positive rate', fontsize=18)
    plt.ylabel('True positive rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.title('ROC curve', fontsize=18)
    plt.legend(loc='best', fontsize=18)
    plt.savefig(name)
    plt.show()

In [5]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = (cm.astype('float') / cm.sum(axis=1)[:, np.newaxis])*100
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=14)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=14)
    plt.yticks(tick_marks, classes, fontsize=14)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 fontsize=14,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.grid(False)
    plt.ylabel('True label', fontsize=14)
    plt.xlabel('Predicted label',fontsize=14)
    plt.savefig(title, bbox_inches='tight')
    plt.show()

In [6]:
def evaluate(X_test,y_test, model, modelname,time):
    #acc = model.score(X_test, y_test)
    #results = model.evaluate(X_test, y_test)
    #acc_r = results[1]
    #print(acc_r)
    y_pred_pr = model.predict(X_test)
    y_pred_cl = np.argmax(y_pred_pr, axis=1)
    print(y_pred_pr[:10])
    print('using probability for having AFIB') 
    y_pred_pr = y_pred_pr[:,1]
    print(y_pred_pr[:10])
   
    print('y_pred_cl')
    print(y_pred_cl[:10])   
    #y_pred = np.argmax(y_pred, axis=1)
    
    print(y_test[:10]) 
    y_test = np.argmax(y_test, axis=1)
    print('y_test')
    print(y_test[:10]) 
    
    ##save predictions and labels to file
    np.save('y_pred', y_pred_pr)
    np.save('y_test', y_test)
    
    tn, fp, fn, tp = confusion_matrix(list(y_test), list(y_pred_cl), labels=[0, 1]).ravel()
    cm = confusion_matrix(list(y_test), list(y_pred_cl), labels=[0, 1])
    #Sensitivity/Recall/TPR
    #tpr= tp/(fn+tp)
    tpr_w = recall_score(y_test, y_pred_cl, average='weighted')
    tpr = recall_score(y_test, y_pred_cl)

    #Specificity/TNR
    tnr = tn/(tn+fp)
    #Precision/Positive Predictive value (weighted)
    #prec = tp/(tp+fp)
    prec = precision_score(y_test, y_pred_cl, average='weighted')
    #FPR
    fpr = fp/(tn+fp)
    #Accuracy
    acc = (tp+tn)/(tp+tn+fp+fn)
    try:
        roc_auc = roc_auc_score(y_test, y_pred_pr)
        ROC_curve(y_test, y_pred_pr, name='ROC_curve')
    except ValueError:
        #pass
        roc_auc = np.nan    
    
    """save confusion matrix as dataframe"""
    confusionDF = pd.DataFrame({'tn': [tn] ,'fp':[fp], 'fn':[fn], 'tp':[tp]}, 
                           columns=['tn','fp', 'fn', 'tp'])
    confusionDF.to_pickle('confusionDF.pkl')

    # non-normalized confusion matrix
    plot_confusion_matrix(cm, classes=['Normal','AFIB'],
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues)
    # normalized confusion matrix
    plot_confusion_matrix(cm, classes=['Normal','AFIB'],
                          normalize=True,
                          title='Confusion matrix_norm',
                          cmap=plt.cm.Blues)
    
    f1 = f1_score(y_test, y_pred_cl, average='weighted')

    toappend = pd.DataFrame({'model': [modelname] ,'acc':[acc], 'tpr':[tpr], 'tnr':[tnr], 'prec':[prec],'fpr':[fpr], 'roc_auc':[roc_auc], 'f1':[f1], 'time': [time]}, 
                           columns=['model','acc', 'tpr', 'tnr', 'prec','fpr','roc_auc','f1', 'time'])
    
    return toappend


In [11]:

def split_train_eval(model, epochsN,BATCH=32,SHUFFLE=True,SPLITS=1):

    metrics = pd.DataFrame({'model':[],'acc':[], 'tpr':[], 'tnr':[], 'prec':[], 'fpr':[], 'roc_auc':[], 'f1':[],'time':[]},
                        columns=['model','acc', 'tpr', 'tnr', 'prec','fpr','roc_auc','f1','time'])
    
    """change seed to 40 from 42"""
    gss = GroupShuffleSplit(n_splits=SPLITS, train_size=.7, random_state=42)
    
    for train_index, test_index in gss.split(X, y, ids):
    
        X_train = np.array(list(itemgetter(*train_index)(X)))
        X_test = np.array(list(itemgetter(*test_index)(X)))
        y_train = np.array(list(itemgetter(*train_index)(y)))
        y_test = np.array(list(itemgetter(*test_index)(y)))
    
        print(f"shape of x_train: {np.shape(X_train)}, type: {type(X_train)}")
        print(f"shape of x_test: {np.shape(X_test)}, type: {type(X_test)}")
        
        
        if (len(X_train)+len(X_test) != len(X)):
            raise TypeError("train and test sets don't add up!")
        
        #get time for start of training. Used in evaluate()
        start = time.time()
        
        model.compile(loss='binary_crossentropy',
            optimizer='adam', metrics = ['accuracy'])

        history = model.fit(X_train, y_train,    
                    epochs=epochsN,
                    batch_size = BATCH,
                    validation_data=(X_test, y_test),
                    shuffle=SHUFFLE,        
                    verbose=1)
        print("fitted")
        pickle.dump(model, open('trained_model.pkl', 'wb'))
        timeToFit =  time.time()-start

        toappend = evaluate(X_test,y_test, model, "combo", timeToFit)

        metrics = pd.concat([metrics, toappend])
        print("appended metrics")

        losscurve(history)
        acccurve(history)
    metrics.to_pickle('metrics.pkl')


In [4]:

def split_train_scale_eval(model, epochsN, func1, BATCH=32):

    metrics = pd.DataFrame({'model':[],'acc':[], 'tpr':[], 'tnr':[], 'prec':[], 'fpr':[], 'roc_auc':[], 'f1':[],'time':[]},
                        columns=['model','acc', 'tpr', 'tnr', 'prec','fpr','roc_auc','f1','time'])
    
    """change seed to 40 from 42"""
    gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=42)
    
    for train_index, test_index in gss.split(X, y, ids):
    
        X_train = np.array(list(itemgetter(*train_index)(X)))
        X_test = np.array(list(itemgetter(*test_index)(X)))
        y_train = np.array(list(itemgetter(*train_index)(y)))
        y_test = np.array(list(itemgetter(*test_index)(y)))
    
        X_train, X_test = scaling(X_train, X_test, func1)
        
        print(f"shape of x_train: {np.shape(X_train)}, type: {type(X_train)}")
        print(f"shape of x_test: {np.shape(X_test)}, type: {type(X_test)}")
        
        
        if (len(X_train)+len(X_test) != len(X)):
            raise TypeError("train and test sets don't add up!")
        
        #get time for start of training. Used in evaluate()
        start = time.time()
        
        model.compile(loss='binary_crossentropy',
            optimizer='adam', metrics = ['accuracy'])

        history = model.fit(X_train, y_train,    
                    epochs=epochsN,
                    batch_size = BATCH,
                    validation_data=(X_test, y_test),
                    shuffle=True,        
                    verbose=1)
        print("fitted")
        timeToFit =  time.time()-start

        toappend = evaluate(X_test,y_test, model, "combo", timeToFit)

        metrics = pd.concat([metrics, toappend])
        print("appended metrics")

        losscurve(history)
        acccurve(history)
        metrics.to_pickle('metrics.pkl')


In [3]:
def foo():
        pass

def scaling(X_TRAIN,X_TEST, func1):    

    # get to 2d shape
    X_TRAIN = np.reshape(X_TRAIN, (X_TRAIN.shape[0], X_TRAIN.shape[1]))
    X_TEST = np.reshape(X_TEST, (X_TEST.shape[0], X_TEST.shape[1]))

    #replace -2 w/ NaN to be ignored in scaling
    X_TRAIN = np.where(X_TRAIN==-2, np.nan, X_TRAIN)
    X_TEST = np.where(X_TEST==-2, np.nan, X_TEST)

    print('TRAIN SHAPE: ' + str(np.shape(X_TRAIN)))
    print('TEST SHAPE: ' + str(np.shape(X_TEST)))
    
    #scale on global level
    scaler = func1().fit(X_TRAIN)
    X_TRAIN = scaler.transform(X_TRAIN)
    #use scaler fit on TRAIN to scale TEST to avoid data leakage
    X_TEST = scaler.transform(X_TEST)

    #replace NaN w/ -2 to be ignored in masking 
    np.nan_to_num(X_TRAIN, copy=False, nan=-2)
    np.nan_to_num(X_TEST, copy=False, nan=-2)

    #get back to 3d shape
    X_TRAIN = np.reshape(X_TRAIN, (X_TRAIN.shape[0], X_TRAIN.shape[1],1))
    X_TEST = np.reshape(X_TEST, (X_TEST.shape[0], X_TEST.shape[1],1))
    
    print('TRAIN SHAPE: ' + str(np.shape(X_TRAIN)))
    print('TEST SHAPE: ' + str(np.shape(X_TEST)))
    
    return X_TRAIN, X_TEST


In [3]:
from sklearn.model_selection import StratifiedKFold

def stratKfold_eval(modelname, EPOCHS, BUILDMODEL, BATCH=32):


    metrics = pd.DataFrame({'model':[],'acc':[], 'tpr':[], 'tnr':[], 'prec':[], 'fpr':[], 'roc_auc':[], 'f1':[],'time':[]}
                           ,columns=['model','acc', 'tpr', 'tnr', 'prec','fpr','roc_auc','f1','time'])

    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    #y_str is y not Onehotencoded
    for i, (train_index, test_index) in enumerate(skf.split(X, y_str)):
            print(f"Fold {i}:")
            print('***')
            print([X[111:116,:5].flatten(), y[111:116,:].flatten(),y_str[111:116]])
            print('***')
            print(f"  Train: index={train_index}")
            print(f"  Test:  index={test_index}")
            X_train = np.array(list(itemgetter(*train_index)(X)))
            X_test = np.array(list(itemgetter(*test_index)(X)))
            y_train = np.array(list(itemgetter(*train_index)(y)))
            y_test = np.array(list(itemgetter(*test_index)(y)))

            print(f"shape of x_train: {np.shape(X_train)}, type: {type(X_train)}")
            print(f"shape of x_test: {np.shape(X_test)}, type: {type(X_test)}")
            
            
            if (len(X_train)+len(X_test) != len(X)):
                raise TypeError("train and test sets don't add up!")

            """build model again at each iteration to reset weigths"""
            model = BUILDMODEL()

            #get time for start of training. Used in evaluate()
            start = time.time()

            model.compile(loss='binary_crossentropy',
                optimizer='adam', metrics = ['accuracy'])

            history = model.fit(X_train, y_train,    
                        epochs=EPOCHS,
                        batch_size = BATCH,
                        validation_data=(X_test, y_test),
                        shuffle=True,        
                        verbose=1)
            print("fitted")
            timeToFit =  time.time()-start

            toappend = evaluate(X_test,y_test, model, modelname, timeToFit)

            metrics = pd.concat([metrics, toappend])
            print("appended metrics")

            #losscurve(history)
            #acccurve(history)
    
    metrics.to_pickle('final_metrics.pkl')




In [3]:
#func is modelbuilding function
def logoloopMask(PATH, func):

    metrics = pd.DataFrame({'model':[],'acc':[], 'tpr':[], 'tnr':[], 'prec':[], 'fpr':[], 'roc_auc':[], 'f1':[],'time':[]},
                        columns=['model','acc', 'tpr', 'tnr', 'prec','fpr','roc_auc','f1','time'])
    history = []
    
    logo = LeaveOneGroupOut()
    logo.get_n_splits(groups=ids)

    for train_index, test_index in logo.split(X, y, ids):
        #print(datetime.datetime.now())
        
        X_train = np.array(list(itemgetter(*train_index)(X)))
        X_test = np.array(list(itemgetter(*test_index)(X)))
        y_train = np.array(list(itemgetter(*train_index)(y)))
        y_test = np.array(list(itemgetter(*test_index)(y)))

        #y_train_all.append(y_train)
        #y_test_all.append(y_test)
        print(X_train.shape)
        print(y_train.shape)
        print(f"shape of y: {np.shape(y_train)}, type of y: {type(y_train)}")
        print(f"shape of x: {np.shape(X_train)}, type of y: {type(y_train)}")

        #Changing name of model to ID of test-individual
        testID =  np.array(list(itemgetter(*test_index)(ids)))[0]
        
        #get time for start of training. Used in evaluate()
        start = time.time()
        
        """build model again at each iteration to reset weigths"""
        model = func()
        
        
        history = model.fit(X_train, y_train,    
                    epochs=50,
                    batch_size = 128,
                    validation_data=(X_test, y_test),
                    verbose=1,)
        print("fitted")
        
        #innerPath='results/logo/'+testID)
        innerPath=PATH+testID


        try:
            os.mkdir(startdir+innerPath)
            print('made dir '+testID)
        except OSError as error:
            print('dir already exists')
            pass                
       
        os.chdir(startdir+innerPath)

        timeToFit =  time.time()-start
        toappend = evaluate(X_test,y_test, model, testID , timeToFit)
        toappend.to_pickle('metrics.pkl')

        metrics = pd.concat([metrics, toappend])
        print("appended metrics")
    
        losscurve(history)
        acccurve(history)
        
    os.chdir(startdir+PATH)    
    metrics.to_pickle('final_metrics.pkl')
    


In [ ]:
def pruning(X_TEST,y_TEST,ids_TEST, ANNOT):
    print(f'N windows in total is {len(annot)}  \n')
    other = np.where( ANNOT == 'Other')
    print('***indices of ***')
    print(np.where( ANNOT == 'Other'))
    #print(other)
    annot = np.delete(ANNOT, other)
    X_test = np.delete(X_TEST, other,0)
    y_test = np.delete(y_TEST, other,0)
    ids_test = np.delete(ids_TEST, other)
    print(f'N windows after removal is {len(annot)}  \n')  
    
    #confirm that arrays are same length and correct shape
    print('***ids***')
    print(np.shape(ids_TEST))
    print('***X***')
    print(np.shape(X_TEST))
    print('***Y***')
    print(np.shape(y_TEST))
    print('***annot***')
    print(np.shape(ANNOT)) 
    return X_TEST,y_TEST,ids_TEST, ANNOT
